<H1>Notes:</H1>

Versuch 1: JsonPATH: https://www.journaldev.com/33265/python-jsonpath-examples (zu kompliziert)

Versuch 2: Pandas Normalize: https://github.com/BindiChen/machine-learning/blob/master/data-analysis/028-pandas-json_normalize/pandas-json_normalize.ipynb?ref=morioh.com&utm_source=morioh.com
- https://pypi.org/project/jsonpath-ng/
- https://pandas-docs.github.io/pandas-docs-travis/reference/api/pandas.io.json.json_normalize.html
- https://alyssasfu.wordpress.com/2015/03/24/working-with-nested-dictionaries-in-python/
- https://towardsdatascience.com/how-to-parse-json-data-with-python-pandas-f84fbd0b1025

---
<h2>Methode 1: def flatten_nested_json_df</h2>

- https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json
- Klappt nicht für Listen in Listen (zB. BaseQuantities)

In [278]:
import json
import pandas as pd
import json
from flatten_json import flatten

with open("C:/Users/DavidRasner/Downloads/Example JSON Input.json") as project_file:    
    data = json.load(project_file)

df = pd.json_normalize(data, record_path=["Objects"])

#flatten JSON

def flatten_nested_json_df(df):

    df = df.reset_index()

    #print(f"original shape: {df.shape}")
    #print(f"original columns: {df.columns}")

    # search for columns to explode/flatten
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()

    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    #print(f"lists: {list_columns}, dicts: {dict_columns}")
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            # print(f"flattening: {col}")
            # explode dictionaries horizontally, adding new columns
            horiz_exploded = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horiz_exploded.index = df.index
            df = pd.concat([df, horiz_exploded], axis=1).drop(columns=[col])
            new_columns.extend(horiz_exploded.columns) # inplace

        for col in list_columns:
            # print(f"exploding: {col}")
            # explode lists vertically, adding new columns
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)

        # check if there are still dict o list fields to flatten
        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()

        # print(f"lists: {list_columns}, dicts: {dict_columns}")

    #print(f"final shape: {df.shape}")
    #print(f"final columns: {df.columns}")
    return df

df_flat = flatten_nested_json_df(df)

df_flat

#df_flat.to_csv("C:/Users/DavidRasner/Desktop/metdata.csv", sep='\t')



,index,$CompositionChildren,Pset_ProjectCommon,Pset_SiteCommon,Pset_BuildingCommon,BaseQuantities,Pset_BuildingStoreyCommon,$Assignments,$Classification.SIA 416,$Classification.SIA d0165,...,$IFC.description.value,$IFC.elevationofrefheight.value,$IFC.elevationofterrain.value,$IFC.elevation.value,$IFC.elevationwithflooring.value,$IFC.interiororexteriorspace.value,$IFC.predefinedtype.value,$IFC.overallheight.value,$IFC.overallwidth.value,$IFC.tag.value
0,0,"[{'compositiontype': {'value': 'ELEMENT'}, 'gl...","[{'BuildingPermitId': {'value': '01.01.2020'},...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"[{'compositiontype': {'value': 'ELEMENT'}, 'de...",NaN,"[{'BuildingHeightLimit': {'value': 18000.0}, '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"[{'compositiontype': {'value': 'ELEMENT'}, 'el...",NaN,NaN,[{'AncillaryFireUse': {'value': 'Kompaktfassad...,NaN,NaN,NaN,NaN,NaN,...,Digitales Bauwerksmodell welches zur Internen ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"[{'compositiontype': {'value': 'ELEMENT'}, 'el...",NaN,NaN,NaN,[{'GrossFloorArea': {'quantitytype': 'areavalu...,"[{'AboveGround': {'value': False}, 'EntranceLe...",NaN,NaN,NaN,...,NaN,NaN,NaN,-3420.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,[{'GrossCeilingArea': {'quantitytype': 'areava...,NaN,[{'globalid': {'value': '3Fn__EKaeHwvQaNFDmXue...,[{'globalid': {'value': '32ui5DKaeHwvQaNFDmXue...,[{'globalid': {'value': '32ui5AKaeHwvQaNFDmXue...,...,NaN,NaN,NaN,NaN,-3420.0,INTERNAL,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,1057,NaN,NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__4KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,1058,NaN,NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '2g97D5nPWHwOAGOaMqxL7...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
1059,1059,NaN,NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__1KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN
1060,1060,NaN,NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__4KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [279]:
df

,$CompositionChildren,$IFC,Pset_ProjectCommon,Pset_SiteCommon,Pset_BuildingCommon,BaseQuantities,Pset_BuildingStoreyCommon,$Assignments,$Classification.SIA 416,$Classification.SIA d0165,...,Pset_DoorCommon,Pset_ShadingDeviceCommon,Pset_BeamCommon,ITUER_Zarge_1fl,ITUER_Rohrrahmen_1fl,ITUER_Rohrrahmen_2fl,ITUER_Zarge_Schiebe,Fenster_3fl_2.1x2.23_Optimiert,Pset_ElementShading,Pset_WindowCommon
0,"[{'compositiontype': {'value': 'ELEMENT'}, 'gl...","[{'$LocalId': {'value': 56}, '$Type': {'value'...","[{'BuildingPermitId': {'value': '01.01.2020'},...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'compositiontype': {'value': 'ELEMENT'}, 'de...","[{'$LocalId': {'value': 61}, '$Type': {'value'...",NaN,"[{'BuildingHeightLimit': {'value': 18000.0}, '...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[{'compositiontype': {'value': 'ELEMENT'}, 'el...","[{'$LocalId': {'value': 73}, '$Type': {'value'...",NaN,NaN,[{'AncillaryFireUse': {'value': 'Kompaktfassad...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'compositiontype': {'value': 'ELEMENT'}, 'el...","[{'$LocalId': {'value': 74}, '$Type': {'value'...",NaN,NaN,NaN,[{'GrossFloorArea': {'quantitytype': 'areavalu...,"[{'AboveGround': {'value': False}, 'EntranceLe...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,"[{'$LocalId': {'value': 93}, '$Type': {'value'...",NaN,NaN,NaN,[{'GrossCeilingArea': {'quantitytype': 'areava...,NaN,[{'globalid': {'value': '3Fn__EKaeHwvQaNFDmXue...,[{'globalid': {'value': '32ui5DKaeHwvQaNFDmXue...,[{'globalid': {'value': '32ui5AKaeHwvQaNFDmXue...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,NaN,"[{'$LocalId': {'value': 471062}, '$Type': {'va...",NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__4KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,NaN,"[{'$LocalId': {'value': 471138}, '$Type': {'va...",NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '2g97D5nPWHwOAGOaMqxL7...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1059,NaN,"[{'$LocalId': {'value': 471198}, '$Type': {'va...",NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__1KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1060,NaN,"[{'$LocalId': {'value': 471256}, '$Type': {'va...",NaN,NaN,NaN,[{'GrossFootprintArea': {'quantitytype': 'area...,NaN,[{'globalid': {'value': '3Fn__4KaeHwvQaNFDmXue...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
<h2>Methode 2: def flatten_json</h2>

- https://towardsdatascience.com/flattening-json-objects-in-python-f5343c794b10
- Beim Qualifier nicht so hilreich

In [273]:
import pandas as pd
import json
from flatten_json import flatten

with open("C:/Users/DavidRasner/Downloads/Example JSON Input.json") as project_file:    
    data = json.load(project_file)

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

flat = flatten_json(data)

pd.json_normalize(flat)

,IfcHeader_IfcFileDescription_description_0,IfcHeader_IfcFileDescription_description_1,IfcHeader_IfcFileDescription_implementation_level,IfcHeader_IfcFileName_author_0,IfcHeader_IfcFileName_authorization,IfcHeader_IfcFileName_name,IfcHeader_IfcFileName_organization,IfcHeader_IfcFileName_originating_system,IfcHeader_IfcFileName_preprocessor_version,IfcHeader_IfcFileName_time_stamp,...,Objects_1061_BaseQuantities_0_GrossVolume_quantitytype,Objects_1061_BaseQuantities_0_GrossVolume_value,Objects_1061_BaseQuantities_0_NetArea_quantitytype,Objects_1061_BaseQuantities_0_NetArea_value,Objects_1061_BaseQuantities_0_NetVolume_quantitytype,Objects_1061_BaseQuantities_0_NetVolume_value,Objects_1061_BaseQuantities_0_Width_quantitytype,Objects_1061_BaseQuantities_0_Width_value,Objects_1061_ExportOptions_0_Export Separate Instances_value,Objects_1061_ExportOptions_0_Export by Components_value
0,"ViewDefinition [CoordinationView_V2.0, SpaceBo...",ExchangeRequirement[Architecture],None,Marc Pancera,None,E:/OneDrive/IttenBrechbuehl/09_ProjPlan/0902_P...,None,None,None,None,...,volumevalue,80.6404,areavalue,322.5616,volumevalue,80.6404,lengthvalue,250.0,False,False


<h2>Methode 3: Flatten 2</h2>
- https://stackoverflow.com/questions/53080998/python-nested-json-import-convert-to-table

In [275]:
import pandas as pd
import json

with open("C:/Users/DavidRasner/Downloads/Example JSON Input.json") as project_file:    
    data = json.load(project_file)

    
def flatten_json(org_json,sep='_'):
    flattened = {}

    def flatten_level(d, prefix=''):
        if type(d) is dict:
            for k in d:
                flatten_level(d[k], prefix + k + sep)
        elif type(d) is list:
            for i,k in enumerate(d):
                flatten_level(k, prefix + str(i) + sep)
        else:
            flattened[prefix[:-1]] = d

    flatten_level(org_json)
    return flattened

flatten_json(data)

{'IfcHeader_IfcFileDescription_description_0': 'ViewDefinition [CoordinationView_V2.0, SpaceBoundary1stLevelAddOnView, QuantityTakeOffAddOnView]',
 'IfcHeader_IfcFileDescription_description_1': 'ExchangeRequirement[Architecture]',
 'IfcHeader_IfcFileDescription_implementation_level': None,
 'IfcHeader_IfcFileName_author_0': 'Marc Pancera',
 'IfcHeader_IfcFileName_authorization': None,
 'IfcHeader_IfcFileName_name': 'E:/OneDrive/IttenBrechbuehl/09_ProjPlan/0902_Produkte/0001_I-B_21_A_ARC_v3.ifc',
 'IfcHeader_IfcFileName_organization': None,
 'IfcHeader_IfcFileName_originating_system': None,
 'IfcHeader_IfcFileName_preprocessor_version': None,
 'IfcHeader_IfcFileName_time_stamp': None,
 'IfcHeader_IfcFileSchema_schema_identifiers': None,
 'Metadata_BuildInfo_GitClean': 'clean',
 'Metadata_BuildInfo_GitRef': 'acd8f283034e94ff6678ada2b11bd221463a926b',
 'Metadata_BuildInfo_GitRefName': 'master',
 'Metadata_BuildInfo_Host': '93f4fd9bfc8d',
 'Metadata_BuildInfo_User': 'build',
 'Metadata_Ori

In [25]:
# -*- coding: utf-8 -*-

import json
import pandas as pd

def main():
	
	#Json in
	filepath = r"C:\Users\DavidRasner\Downloads\Example JSON Input.json"
	filein = open(filepath)
	metadata = json.load(filein)
	
	CreateDataframe(metadata)
	
	pass

def CreateDataframe(metadata):
	
	strGroupFieldSeparator = "#"
	strListEntrySeparator = "; "
	
	#Nested dict with all groupnames and fieldnames
	#$IFC and $Entity are hardcoded, so they stand at the beginning of the dict
	dictFields = {}
	dictFields["$IFC"] = {}
	dictFields["$Entity"] = {}
	
	#Determine all groupname/fieldname combinations that will become the columns
	for i in range(len(metadata["Objects"])):
		for groupname in metadata["Objects"][i]:
			
			if not groupname in dictFields:
				dictFields[groupname] = {}
			
			for fieldname in metadata["Objects"][i][groupname][0]:
				
				if not fieldname in dictFields[groupname]:
					dictFields[groupname][fieldname] = None
	
	#Total number of entities (=rows)
	intEntities = len(metadata["Objects"])
	
	#Create dict for dataframe with keys (=columns) from groupname.fieldname combinations
	#Create lists with intEntities empty fields as values
	dictDF = {}
	for groupname in dictFields:
		for fieldname in dictFields[groupname]:
			dictDF[groupname + strGroupFieldSeparator + fieldname] = [None]*intEntities
	
	#Overwrite empty list entries with metadata values from json data
	for i in range(len(metadata["Objects"])):
		for groupname in metadata["Objects"][i]:
			for fieldname in metadata["Objects"][i][groupname][0]:
				
				colName = groupname + strGroupFieldSeparator  + fieldname
				value = metadata["Objects"][i][groupname][0][fieldname]["value"]
				
				if type(value) is str:
					pass
				elif type(value) is bool:
					pass
				elif type(value) is int:
					pass
				elif type(value) is float:
					pass
				elif type(value) is type(None):
					pass
				elif type(value) is list:
					value = strListEntrySeparator.join(str(s) for s in value)
				
				dictDF[colName][i] = value
	
	#Convert dictDF to dataframe
	
	pass

if __name__ == "__main__": main()

In [34]:
df = pd.DataFrame(data=CreateDataframe())
df

TypeError: CreateDataframe() missing 1 required positional argument: 'metadata'

NameError: name 'dictDF' is not defined